In [1]:
import pandas as pd

In [6]:
applications = pd.read_csv('../data/2022_public_lar_csv.csv', low_memory=False)
applications

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2022,RVDPPPGHCGZ40J4VQ731,26420,TX,48201.0,4.820155e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6907,71.83,90800,80.0,1409,2002,28
1,2022,RVDPPPGHCGZ40J4VQ731,19124,TX,48085.0,4.808503e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6001,39.96,97400,150.0,1255,1431,14
2,2022,RVDPPPGHCGZ40J4VQ731,40900,CA,6017.0,6.017030e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2703,39.92,102400,60.0,478,1631,51
3,2022,RVDPPPGHCGZ40J4VQ731,25540,CT,9013.0,9.013538e+09,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,3948,8.61,113200,121.0,1271,1507,42
4,2022,RVDPPPGHCGZ40J4VQ731,12580,MD,24510.0,2.451015e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,3249,92.34,116100,38.0,444,1500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16080205,2022,RVDPPPGHCGZ40J4VQ731,12060,GA,13117.0,1.311713e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,4745,34.77,95700,166.0,1380,1497,17
16080206,2022,RVDPPPGHCGZ40J4VQ731,29820,NV,32003.0,3.200300e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,NaN,NaN,NaN,5249,84.82,81700,50.0,388,1007,36
16080207,2022,RVDPPPGHCGZ40J4VQ731,39340,UT,49049.0,4.904901e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5884,15.89,96000,110.0,1188,1327,20
16080208,2022,RVDPPPGHCGZ40J4VQ731,29820,NV,32003.0,3.200300e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,6749,66.14,81700,105.0,1022,2223,16


In [7]:
applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16080210 entries, 0 to 16080209
Data columns (total 99 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   activity_year                             int64  
 1   lei                                       object 
 2   derived_msa_md                            int64  
 3   state_code                                object 
 4   county_code                               float64
 5   census_tract                              float64
 6   conforming_loan_limit                     object 
 7   derived_loan_product_type                 object 
 8   derived_dwelling_category                 object 
 9   derived_ethnicity                         object 
 10  derived_race                              object 
 11  derived_sex                               object 
 12  action_taken                              int64  
 13  purchaser_type                            int64  
 14  

In [8]:
applications['combined_loan_to_value_ratio'].value_counts(dropna=False)

NaN        5279299
80.0       1156495
95.0        577277
96.5        439174
90.0        374199
            ...   
64.1521          1
81.4743          1
67.0912          1
39.4553          1
278.69           1
Name: combined_loan_to_value_ratio, Length: 167445, dtype: int64